In [ ]:
# scRNA-seq Analysis Pipeline

This notebook performs:
- QC and filtering
- Normalization and HVG selection
- Clustering and UMAP
- Batch integration using Harmony

## Requirements
- scanpy
- harmonypy

In [ ]:
DATA_PATH = "/your/data/path"   # e.g. "/home/user/data"
FILE = "file_name"              # e.g. "filtered_feature_bc_matrix"

import os

def load_10x(data_path, file_name):
    return sc.read_10x_mtx(
        path=os.path.join(data_path, file_name),
        var_names="gene_symbols",
        cache=True
    )

colon_WT = load_10x(DATA_PATH, FILE)
colon_KO = load_10x(DATA_PATH, FILE)
spleen_WT = load_10x(DATA_PATH, FILE)
spleen_KO = load_10x(DATA_PATH, FILE)

In [ ]:
# Check quality
for adata in [colon_WT, colon_KO, spleen_WT, spleen_KO]:
    adata.var["mt"] = adata.var_names.str.startswith("mt-")
    sc.pp.calculate_qc_metrics(
        adata,
        qc_vars=["mt"],
        percent_top=None,
        log1p=False,
        inplace=True,
    )
    sc.pl.violin(
        adata,
        ["n_genes_by_counts", "total_counts", "pct_counts_mt"],
        jitter=0.4,
        multi_panel=True,
    )

In [ ]:
# Processing
datasets = {
    "colon_WT": colon_WT,
    "colon_KO": colon_KO,
    "spleen_WT": spleen_WT,
    "spleen_KO": spleen_KO,
}

for name, adata in datasets.items():

    print(f"Before processing: {name}")
    print(adata)

    adata.var["mt"] = adata.var_names.str.startswith("mt-")
    sc.pp.calculate_qc_metrics(adata, qc_vars=["mt"], inplace=True)

    sc.pp.filter_cells(adata, min_genes=200)
    sc.pp.filter_genes(adata, min_cells=3)

    adata = adata[adata.obs["pct_counts_mt"] < 5].copy()

    sc.pp.normalize_total(adata, target_sum=1e4)
    sc.pp.log1p(adata)

    adata.raw = adata

    sc.pp.highly_variable_genes(adata, n_top_genes=2000)
    sc.pp.scale(adata, max_value=10)

    print(f"After processing: {name}")
    print(adata)

    datasets[name] = adata

In [ ]:
# Clustering
for adata in [colon_WT, colon_KO, spleen_WT, spleen_KO]:
    sc.tl.pca(adata)
    sc.pp.neighbors(adata, use_rep="X_pca")
    sc.tl.leiden(adata, flavor="igraph", n_iterations=2)
    sc.tl.umap(adata)
    sc.pl.umap(adata, color=["leiden"])

In [ ]:
# Integration colon
colon_WT = datasets["colon_WT"]
colon_KO = datasets["colon_KO"]

colon_WT.obs["batch"] = "colon_WT"
colon_KO.obs["batch"] = "colon_KO"

colon_data = colon_WT.concatenate(
    colon_KO,
    batch_key="batch",
    batch_categories=["colon_WT", "colon_KO"],
)

colon_data_hm = hm.run_harmony(
    colon_data.obsm["X_pca"],
    colon_data.obs,
    "batch",
)

colon_data.obsm["X_pca_harmony"] = colon_data_hm.Z_corr.T

sc.pp.neighbors(colon_data, use_rep="X_pca_harmony")
sc.tl.umap(colon_data)
sc.tl.leiden(colon_data)

In [ ]:
# Integration spleen
spleen_WT = datasets["spleen_WT"]
spleen_KO = datasets["spleen_KO"]

spleen_WT.obs["batch"] = "spleen_WT"
spleen_KO.obs["batch"] = "spleen_KO"

spleen_data = spleen_WT.concatenate(
    spleen_KO,
    batch_key="batch",
    batch_categories=["spleen_WT", "spleen_KO"],
)

spleen_data_hm = hm.run_harmony(
    spleen_data.obsm["X_pca"],
    spleen_data.obs,
    "batch",
)

spleen_data.obsm["X_pca_harmony"] = spleen_data_hm.Z_corr.T

sc.pp.neighbors(spleen_data, use_rep="X_pca_harmony")
sc.tl.umap(spleen_data)
sc.tl.leiden(spleen_data)

In [ ]:
# Check integration
sc.pl.umap(colon_data, color=['batch'], title='Colon', size=20)
sc.pl.umap(spleen_data, color=['batch'], title='Spleen', size=20)